In [1]:
# !pip3 install folium
# !pip3 install folium --upgrade

In [2]:
import folium

## 기본 지도 나타내기

#### Folium 
- https://github.com/python-visualization/folium

In [3]:
# _01 folium 지도 생성
folium.Map(location=[0,0])

## 위도와 경도

In [4]:
# _02 위도, 경도를 입력해 지도의 처음 위치 설정
folium.Map(location=[37.573050, 126.979189])

## 지도 옵션 변경하기

- 지도크기: width, height
- zoom level : 0 ~ 18 (default 10)
- zoom_start / max_zoom / min_zoom / zoom_control
- tiles (default: OpenStreetMap)

In [5]:
# _03 지도 옵션
folium.Map(
    location=[37.573050, 126.979189],
    width=700, height=500,
#     min_zoom=9,
#     max_zoom=12,
    zoom_start=11,
    zoom_control=False
)

## 타일맵 적용하기

In [6]:
# _04 Tile Map Test
# Default Tile - OpenStreetMap

# “OpenStreetMap”
# “Mapbox Bright” (Limited levels of zoom for free tiles)
# “Mapbox Control Room” (Limited levels of zoom for free tiles)
# “Stamen” (Terrain, Toner, and Watercolor)
# “Cloudmade” (Must pass API key)
# “Mapbox” (Must pass API key)
# “CartoDB” (positron and dark_matter)

folium.Map(
    location=[37.573050, 126.979189],
    tiles='Stamen Terrain',
    width=700, height=500,
    min_zoom=9,
    max_zoom=12,
    zoom_start=11
)

## 지도에 Marker 올리기

In [7]:
# _05 Marker 실습
marker_map = folium.Map(
    location=[37.573050, 126.979189],
    tiles='CartoDB positron',
    width=700, height=500,
    zoom_start=11
)

# coustom_icon = folium.features.CustomIcon('./image/icons/general.png', icon_size=(38, 60))

# 스타벅스 논현역사거리점
folium.Marker(
    location=[37.510221, 127.022268], 
    popup='<h4>강남구</h4>', 
    tooltip='논현역사거리',
    icon=folium.Icon(color='red', icon='info-sign')
#     icon=coustom_icon
).add_to(marker_map)

marker_map

In [8]:
# _06 서울시 스타벅스 매장 지도 시각화
import geopandas as gpd
_seoul_sb = './maps/final/seoul_starbucks.geojson'
seoul_sb_gdf = gpd.read_file(_seoul_sb)
seoul_sb_gdf.info()
seoul_sb_gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 5 columns):
STORE_NAME    495 non-null object
STORE_TYPE    495 non-null object
SIG_CD        495 non-null object
SIG_KOR_NM    495 non-null object
geometry      495 non-null object
dtypes: object(5)
memory usage: 19.4+ KB


,STORE_NAME,STORE_TYPE,SIG_CD,SIG_KOR_NM,geometry
0,역삼아레나빌딩,general,11680,강남구,POINT (127.043069 37.501087)
1,논현역사거리,general,11680,강남구,POINT (127.022223 37.510178)
2,대치대원빌딩R,reserve,11680,강남구,POINT (127.062583 37.494668)
3,삼성역섬유센터R,reserve,11680,강남구,POINT (127.060651 37.50775)
4,압구정R,reserve,11680,강남구,POINT (127.033061 37.5273669)


In [9]:
# _07 GeoDataFrame을 geojson으로 변환하기
gjson = seoul_sb_gdf.to_json()
gjson

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"SIG_CD": "11680", "SIG_KOR_NM": "\\uac15\\ub0a8\\uad6c", "STORE_NAME": "\\uc5ed\\uc0bc\\uc544\\ub808\\ub098\\ube4c\\ub529", "STORE_TYPE": "general"}, "geometry": {"type": "Point", "coordinates": [127.043069, 37.501087]}}, {"id": "1", "type": "Feature", "properties": {"SIG_CD": "11680", "SIG_KOR_NM": "\\uac15\\ub0a8\\uad6c", "STORE_NAME": "\\ub17c\\ud604\\uc5ed\\uc0ac\\uac70\\ub9ac", "STORE_TYPE": "general"}, "geometry": {"type": "Point", "coordinates": [127.022223, 37.510178]}}, {"id": "2", "type": "Feature", "properties": {"SIG_CD": "11680", "SIG_KOR_NM": "\\uac15\\ub0a8\\uad6c", "STORE_NAME": "\\ub300\\uce58\\ub300\\uc6d0\\ube4c\\ub529R", "STORE_TYPE": "reserve"}, "geometry": {"type": "Point", "coordinates": [127.062583, 37.494668]}}, {"id": "3", "type": "Feature", "properties": {"SIG_CD": "11680", "SIG_KOR_NM": "\\uac15\\ub0a8\\uad6c", "STORE_NAME": "\\uc0bc\\uc131\\uc5ed\\uc12c\\uc720\\uc13c\

In [10]:
# _08 Folium지도에 geojson 올려보기
viz_map_1 = folium.Map(
    location=[37.573050, 126.979189],
    tiles='CartoDB positron',
    width=700, height=500,
    zoom_start=11
)

folium.GeoJson(gjson).add_to(viz_map_1)
viz_map_1

In [11]:
# _09 스타벅스 매장 타입별로 서클 마커 그려보기
viz_map_2 = folium.Map(
    location=[37.573050, 126.979189],
    tiles='CartoDB positron',
    zoom_start=11
)

for idx in seoul_sb_gdf.index:
    # 위도/경도 값 가져오기
    point = seoul_sb_gdf.at[idx, 'geometry']
    lat = point.y
    lng = point.x
#     print(point.x)
#     print(point.y)
    # 스타벅스 매장 타입
    store_type = seoul_sb_gdf.at[idx, 'STORE_TYPE']
#     print(store_type)
    
    fillColor = ''
    # 매장 타입별로 색상 변경하기
    if store_type == 'general':
        fillColor = 'green'
    elif store_type == 'reserve':
        fillColor = 'red'
    elif store_type == 'generalDT':
        fillColor = 'orange'

    folium.CircleMarker(
        location=[lat, lng],
        color='grey', 
        fill_color=fillColor, 
        fill_opacity=1,
        weight=0.8,
        radius=3
    ).add_to(viz_map_2)

# folium 지도 실행
viz_map_2

In [12]:
# _10 서울시 시군구 통계 지도 시각화
_seoul_sgg = './maps/final/seoul_sgg_stat.geojson'
seoul_sgg_stat = gpd.read_file(_seoul_sgg)
seoul_sgg_stat.info()
seoul_sgg_stat.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 10 columns):
SGG_CODE      25 non-null object
SGG_NM        25 non-null object
SEDAE         25 non-null int64
SEDAE_INGU    25 non-null float64
TOTAL_POP     25 non-null int64
MALE_POP      25 non-null int64
FEMALE_POP    25 non-null int64
65_OVER       25 non-null int64
COUNT         25 non-null int64
geometry      25 non-null object
dtypes: float64(1), int64(6), object(3)
memory usage: 2.0+ KB


,SGG_CODE,SGG_NM,SEDAE,SEDAE_INGU,TOTAL_POP,MALE_POP,FEMALE_POP,65_OVER,COUNT,geometry
0,11680,강남구,231612,54.29,561052,268941,292111,65060,75,"POLYGON ((127.1110354206454 37.48293504402488,..."
1,11740,강동구,177407,45.24,440359,218699,221660,56161,13,"POLYGON ((127.1458000886755 37.52198996684139,..."
2,11305,강북구,142533,29.67,328002,160252,167750,56530,5,"POLYGON ((127.0220295465139 37.6122870292313, ..."
3,11500,강서구,254257,47.24,608255,296175,312080,76032,14,"POLYGON ((126.8831662843492 37.55446522843518,..."
4,11620,관악구,255352,42.12,520929,262006,258923,70046,10,"POLYGON ((126.9704905925551 37.4753683309936, ..."


In [15]:
# _11 geojson만들기
sgg_geojson = seoul_sgg_stat.to_json()

In [16]:
# _12 행정경계 지도 그리기
def style_function(feature):
    return {
        'opacity': 0.7,
        'weight': 1,
        'fillOpacity':0.1,
        'color': 'black',
        'fillColor': 'yellow',
    }

viz_map_3 = folium.Map(
    location=[37.573050, 126.979189],
    tiles='CartoDB positron',
    zoom_start=11
)

folium.GeoJson(
    sgg_geojson,
    style_function=style_function,
).add_to(viz_map_3)

for idx in seoul_sb_gdf.index:
    # 위도/경도 값 가져오기
    point = seoul_sb_gdf.at[idx, 'geometry']
    lat = point.y
    lng = point.x
    # 스타벅스 매장 타입
    store_type = seoul_sb_gdf.at[idx, 'STORE_TYPE']
    
    fillColor = ''
    # 매장 타입별로 색상 변경하기
    if store_type == 'general':
        fillColor = 'green'
    elif store_type == 'reserve':
        fillColor = 'red'
    elif store_type == 'generalDT':
        fillColor = 'orange'

    folium.CircleMarker(
        location=[lat, lng],
        color='grey', 
        fill_color=fillColor, 
        fill_opacity=1,
        weight=0.8,
        radius=3
    ).add_to(viz_map_3)
    
viz_map_3

In [20]:
# _13 시군구별 중심점 그려보기
seoul_sgg_stat['centoroid'] = seoul_sgg_stat.centroid
seoul_sgg_stat.head()

,SGG_CODE,SGG_NM,SEDAE,SEDAE_INGU,TOTAL_POP,MALE_POP,FEMALE_POP,65_OVER,COUNT,geometry,centoroid
0,11680,강남구,231612,54.29,561052,268941,292111,65060,75,"POLYGON ((127.1110354206454 37.48293504402488,...",POINT (127.0626996922278 37.49718912235008)
1,11740,강동구,177407,45.24,440359,218699,221660,56161,13,"POLYGON ((127.1458000886755 37.52198996684139,...",POINT (127.1468154389302 37.5511696828861)
2,11305,강북구,142533,29.67,328002,160252,167750,56530,5,"POLYGON ((127.0220295465139 37.6122870292313, ...",POINT (127.0111014848345 37.64357998452166)
3,11500,강서구,254257,47.24,608255,296175,312080,76032,14,"POLYGON ((126.8831662843492 37.55446522843518,...",POINT (126.8243347647332 37.56113871888017)
4,11620,관악구,255352,42.12,520929,262006,258923,70046,10,"POLYGON ((126.9704905925551 37.4753683309936, ...",POINT (126.9460094623881 37.46715606159064)


In [21]:
# _14 통계지도 만들기 (단계구분도 + 버블지도)
choropleth_map = folium.Map(
    location=[37.573050, 126.979189],
    tiles='CartoDB dark_matter',
    min_zoom=10,
    max_zoom=12,
    zoom_start=11
)

folium.Choropleth(
    geo_data=sgg_geojson,
    name='choropleth',
    data=seoul_sgg_stat,
    columns=['SGG_CODE', 'TOTAL_POP'],
    key_on='feature.properties.SGG_CODE',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
).add_to(choropleth_map)

mean = seoul_sgg_stat.COUNT.mean()
for idx in seoul_sgg_stat.index:
    point = seoul_sgg_stat.at[idx, 'centoroid']
    lat = point.y
    lng = point.x
    count = seoul_sgg_stat.at[idx, 'COUNT']
    name = seoul_sgg_stat.at[idx, 'SGG_NM']
    
    if count == 0:
        continue
    elif count > mean:
        fillColor = '#FF0000'
    else:
        fillColor = '#CCFF33'
    
    folium.CircleMarker(
        location=[lat, lng], 
        tooltip=name,
        color='#FFFF00', 
        fill_color=fillColor, 
        fill_opacity=0.7,
        weight=1.5,
        radius=count/2
    ).add_to(choropleth_map)

choropleth_map